In [1]:
# Join cutomers, foods, week1 and week2 sales

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

23/01/31 15:47:51 WARN Utils: Your hostname, Presidios-MacBook-Pro-10.local resolves to a loopback address: 127.0.0.1; using 182.11.190.64 instead (on interface en0)
23/01/31 15:47:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/31 15:47:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
customers = spark.read.format("csv").option("header","true").load("Restaurant - Customers.csv")
foods = spark.read.format("csv").option("header","true").load("Restaurant - Foods.csv")
week1Sales = spark.read.format("csv").option("header","true").load("Restaurant - Week 1 Sales.csv")
week2Sales = spark.read.format("csv").option("header","true").load("Restaurant - Week 2 Sales.csv")

In [4]:
customers.show()

+---+----------+---------+------+-------------+--------------------+
| ID|First Name|Last Name|Gender|      Company|          Occupation|
+---+----------+---------+------+-------------+--------------------+
|  1|    Joseph|  Perkins|  Male|      Dynazzy|Community Outreac...|
|  2|  Jennifer|  Alvarez|Female|         DabZ|Senior Quality En...|
|  3|     Roger|    Black|  Male|      Tagfeed|   Account Executive|
|  4|    Steven|    Evans|  Male|         Fatz|    Registered Nurse|
|  5|      Judy| Morrison|Female|      Demivee|     Legal Assistant|
|  6|    Amanda|   Howell|Female|Thoughtbridge|    Dental Hygienist|
|  7|    Johnny|     Reid|  Male|  Babblestorm|   Account Executive|
|  8|   Frances|    Adams|Female|     Dabshots|       Developer III|
|  9|   Gregory|  Chapman|  Male|         Eayo|    Junior Executive|
| 10|    Steven|     Ryan|  Male|      Twinder|Community Outreac...|
| 11|   Michael| Reynolds|  Male|       Meevee|    Dental Hygienist|
| 12|      Lois|  Watkins|Female| 

In [5]:
foods.show()

+-------+----------+-----+
|Food ID| Food Item|Price|
+-------+----------+-----+
|      1|     Sushi| 3.99|
|      2|   Burrito| 9.99|
|      3|      Taco| 2.99|
|      4|Quesadilla| 4.25|
|      5|     Pizza| 2.49|
|      6|     Pasta|13.99|
|      7|     Steak|24.99|
|      8|     Salad|11.25|
|      9|     Donut| 0.99|
|     10|     Drink| 1.75|
+-------+----------+-----+



In [6]:
week1Sales.show()

+-----------+-------+
|Customer ID|Food ID|
+-----------+-------+
|        537|      9|
|         97|      4|
|        658|      1|
|        202|      2|
|        155|      9|
|        213|      8|
|        600|      1|
|        503|      5|
|         71|      3|
|        174|      3|
|        961|      9|
|        966|      5|
|        641|      4|
|        288|      2|
|        149|      4|
|        954|      2|
|        147|      9|
|        155|      1|
|        550|      6|
|        101|      7|
+-----------+-------+
only showing top 20 rows



In [7]:
week2Sales.show()

+-----------+-------+
|Customer ID|Food ID|
+-----------+-------+
|        688|     10|
|        813|      7|
|        495|     10|
|        189|      5|
|        267|      3|
|        310|      5|
|        761|      2|
|        443|      5|
|        729|      9|
|        741|      8|
|        847|      6|
|        156|     10|
|        550|      7|
|        620|     10|
|        272|      4|
|        511|      2|
|          8|      6|
|        534|      7|
|        909|      6|
|        732|      9|
+-----------+-------+
only showing top 20 rows



In [8]:
sales = week1Sales.union(week2Sales)

In [9]:
sales.show()

+-----------+-------+
|Customer ID|Food ID|
+-----------+-------+
|        537|      9|
|         97|      4|
|        658|      1|
|        202|      2|
|        155|      9|
|        213|      8|
|        600|      1|
|        503|      5|
|         71|      3|
|        174|      3|
|        961|      9|
|        966|      5|
|        641|      4|
|        288|      2|
|        149|      4|
|        954|      2|
|        147|      9|
|        155|      1|
|        550|      6|
|        101|      7|
+-----------+-------+
only showing top 20 rows



In [10]:
sales.count()

500

In [11]:
salesAndFoods = sales.join(foods, sales['Food ID'] == foods['Food ID'], "inner").drop(foods['Food ID'])

In [12]:
salesAndFoods.show()

+-----------+-------+----------+-----+
|Customer ID|Food ID| Food Item|Price|
+-----------+-------+----------+-----+
|        537|      9|     Donut| 0.99|
|         97|      4|Quesadilla| 4.25|
|        658|      1|     Sushi| 3.99|
|        202|      2|   Burrito| 9.99|
|        155|      9|     Donut| 0.99|
|        213|      8|     Salad|11.25|
|        600|      1|     Sushi| 3.99|
|        503|      5|     Pizza| 2.49|
|         71|      3|      Taco| 2.99|
|        174|      3|      Taco| 2.99|
|        961|      9|     Donut| 0.99|
|        966|      5|     Pizza| 2.49|
|        641|      4|Quesadilla| 4.25|
|        288|      2|   Burrito| 9.99|
|        149|      4|Quesadilla| 4.25|
|        954|      2|   Burrito| 9.99|
|        147|      9|     Donut| 0.99|
|        155|      1|     Sushi| 3.99|
|        550|      6|     Pasta|13.99|
|        101|      7|     Steak|24.99|
+-----------+-------+----------+-----+
only showing top 20 rows



In [13]:
salesAndFoods.columns

['Customer ID', 'Food ID', 'Food Item', 'Price']

In [14]:
result = customers.join(salesAndFoods, customers['ID'] == salesAndFoods['Customer ID'], "inner").drop(salesAndFoods['Customer ID'])

In [15]:
result.show()

+---+----------+---------+------+------------+--------------------+-------+----------+-----+
| ID|First Name|Last Name|Gender|     Company|          Occupation|Food ID| Food Item|Price|
+---+----------+---------+------+------------+--------------------+-------+----------+-----+
|537|    Cheryl|  Carroll|Female|    Zoombeat|    Registered Nurse|      9|     Donut| 0.99|
| 97|    Amanda|  Watkins|Female|         Ozu| Account Coordinator|      4|Quesadilla| 4.25|
|658|   Patrick|     Webb|  Male|   Browsebug|Community Outreac...|      1|     Sushi| 3.99|
|202|     Louis| Campbell|  Male|   Rhynoodle|Account Represent...|      2|   Burrito| 9.99|
|155|   Carolyn|     Diaz|Female|    Gigazoom|Database Administ...|      9|     Donut| 0.99|
|213|     Keith|   Foster|  Male|   Gigashots|  VP Quality Control|      8|     Salad|11.25|
|600|    Sandra|   Bailey|Female|   Jaxnation|     Sales Associate|      1|     Sushi| 3.99|
|503|    Justin|  Daniels|  Male|        Oloo|  Environmental Tech|   

In [16]:
# Find Customers who have came more than 1 time combing Week1 and Week 2  in descending

In [17]:
customersVisitCount = result.groupBy('ID').count().orderBy('count', ascending = False)

In [18]:
customersVisitCount.show()

+---+-----+
| ID|count|
+---+-----+
| 77|    5|
|495|    4|
|343|    4|
|503|    3|
|155|    3|
|922|    3|
|100|    3|
|937|    3|
|310|    3|
|304|    3|
|909|    3|
|331|    3|
| 21|    3|
|253|    3|
|798|    3|
|761|    3|
|578|    3|
|250|    2|
| 51|    2|
| 71|    2|
+---+-----+
only showing top 20 rows



In [19]:
customersVisitCount.join(customers, customersVisitCount['ID'] == customers['ID'], "left").drop(customers['ID']).show()

+-----+---+----------+---------+------+----------+--------------------+
|count| ID|First Name|Last Name|Gender|   Company|          Occupation|
+-----+---+----------+---------+------+----------+--------------------+
|    5| 77|      Lori|  Edwards|Female|   Dabvine|Sales Representative|
|    4|343|       Joe|    Price|  Male|  Fanoodle|   Marketing Manager|
|    4|495|   Deborah|   Little|Female|Babbleblab|       VP Accounting|
|    3|155|   Carolyn|     Diaz|Female|  Gigazoom|Database Administ...|
|    3|310|     Bobby|     Sims|  Male| Skynoodle|   Marketing Manager|
|    3|503|    Justin|  Daniels|  Male|      Oloo|  Environmental Tech|
|    3|100|      Lois|   Howard|Female|     Voomm|Systems Administr...|
|    3|761| Stephanie|    Olson|Female| Rhynoodle|    Web Developer IV|
|    3|798|     Kelly|   Taylor|Female|     Meejo|     Design Engineer|
|    3|578|    Donald| Matthews|  Male|  Fiveclub|Physical Therapy ...|
|    3|304|  Margaret|     Cruz|Female| Jaxnation|Automation Spe

In [20]:
# Generate Bill Amount for customers eaten food separately for week1 and week2
# Expected OP columns : CustID || Cust name|| Food Names|| Week 1 Bill amount || Week 2 Bill amount

In [21]:
# Join Food and Week 1 Sales
# Join Food and Week 2 Sales

week1SalesAndFoods = week1Sales.join(foods, week1Sales['Food ID'] == foods['Food ID'], "inner").drop(foods['Food ID'])
week2SalesAndFoods = week2Sales.join(foods, week2Sales['Food ID'] == foods['Food ID'], "inner").drop(foods['Food ID'])

In [22]:
week1SalesAndFoods.show()

+-----------+-------+----------+-----+
|Customer ID|Food ID| Food Item|Price|
+-----------+-------+----------+-----+
|        537|      9|     Donut| 0.99|
|         97|      4|Quesadilla| 4.25|
|        658|      1|     Sushi| 3.99|
|        202|      2|   Burrito| 9.99|
|        155|      9|     Donut| 0.99|
|        213|      8|     Salad|11.25|
|        600|      1|     Sushi| 3.99|
|        503|      5|     Pizza| 2.49|
|         71|      3|      Taco| 2.99|
|        174|      3|      Taco| 2.99|
|        961|      9|     Donut| 0.99|
|        966|      5|     Pizza| 2.49|
|        641|      4|Quesadilla| 4.25|
|        288|      2|   Burrito| 9.99|
|        149|      4|Quesadilla| 4.25|
|        954|      2|   Burrito| 9.99|
|        147|      9|     Donut| 0.99|
|        155|      1|     Sushi| 3.99|
|        550|      6|     Pasta|13.99|
|        101|      7|     Steak|24.99|
+-----------+-------+----------+-----+
only showing top 20 rows



In [23]:
week2SalesAndFoods.show()

+-----------+-------+----------+-----+
|Customer ID|Food ID| Food Item|Price|
+-----------+-------+----------+-----+
|        688|     10|     Drink| 1.75|
|        813|      7|     Steak|24.99|
|        495|     10|     Drink| 1.75|
|        189|      5|     Pizza| 2.49|
|        267|      3|      Taco| 2.99|
|        310|      5|     Pizza| 2.49|
|        761|      2|   Burrito| 9.99|
|        443|      5|     Pizza| 2.49|
|        729|      9|     Donut| 0.99|
|        741|      8|     Salad|11.25|
|        847|      6|     Pasta|13.99|
|        156|     10|     Drink| 1.75|
|        550|      7|     Steak|24.99|
|        620|     10|     Drink| 1.75|
|        272|      4|Quesadilla| 4.25|
|        511|      2|   Burrito| 9.99|
|          8|      6|     Pasta|13.99|
|        534|      7|     Steak|24.99|
|        909|      6|     Pasta|13.99|
|        732|      9|     Donut| 0.99|
+-----------+-------+----------+-----+
only showing top 20 rows



In [24]:
# Type cast Price column to double

week1SalesAndFoods = week1SalesAndFoods.withColumn('Price', week1SalesAndFoods['Price'].cast('double'))
week2SalesAndFoods = week2SalesAndFoods.withColumn('Price', week2SalesAndFoods['Price'].cast('double'))

In [25]:
# Union Week1 and Week2 sales as a single dataframe to get the list of food items ordered by a customer
from pyspark.sql.functions import collect_set
from pyspark.sql.functions import sum

week1And2Sales = week1SalesAndFoods.union(week2SalesAndFoods)
week1And2Sales = week1And2Sales.groupBy('Customer ID').agg(collect_set('Food Item').alias('Food Items'))

# Calculate bill amount of a customer in week1 and week2
week1SalesAndFoodsSum = week1SalesAndFoods.groupBy('Customer ID').agg(sum('Price').alias('Week 1 Bill amount'))
week2SalesAndFoodsSum = week2SalesAndFoods.groupBy('Customer ID').agg(sum('Price').alias('Week 2 Bill amount'))

In [26]:
week1SalesAndFoodsSum.show()

+-----------+------------------+
|Customer ID|Week 1 Bill amount|
+-----------+------------------+
|        296|              0.99|
|        919|              1.75|
|        926|             13.99|
|         51|             11.74|
|        574|              1.75|
|        475|              1.75|
|        307|              0.99|
|        747|              2.49|
|        462|              2.99|
|        282|             24.99|
|        155|              4.98|
|        483|             13.99|
|        101|             24.99|
|        433|             24.99|
|        595|              2.99|
|        961|              0.99|
|        351|              1.75|
|        323|              3.99|
|        138|              4.25|
|        419|             24.99|
+-----------+------------------+
only showing top 20 rows



In [27]:
week1And2Sales.show()

+-----------+--------------------+
|Customer ID|          Food Items|
+-----------+--------------------+
|         10|           [Burrito]|
|        100|[Quesadilla, Stea...|
|       1000|           [Burrito]|
|        101| [Quesadilla, Steak]|
|        107|           [Burrito]|
|        108|              [Taco]|
|        110|             [Salad]|
|        111|              [Taco]|
|        114|             [Pizza]|
|        117|             [Pizza]|
|        121|              [Taco]|
|        122|    [Burrito, Drink]|
|        123|             [Pasta]|
|        127|             [Sushi]|
|         13|           [Burrito]|
|        131|             [Sushi]|
|        132|             [Salad]|
|        134|              [Taco]|
|        136|             [Sushi]|
|        138|        [Quesadilla]|
+-----------+--------------------+
only showing top 20 rows



In [28]:
# Join week 1 bill amount column to customers table
customersWeek1Sales = customers.join( \
                                   week1SalesAndFoodsSum, \
                                   customers['ID'] == week1SalesAndFoodsSum['Customer ID'], \
                                   "inner" \
                               ).drop(week1SalesAndFoodsSum['Customer ID'])

# Join week 2 bill amount to previous dataframe
customersWeek1And2Sales = customersWeek1Sales.join( \
                                                   week2SalesAndFoodsSum, \
                                                   customersWeek1Sales['ID'] == week2SalesAndFoodsSum['Customer ID'], \
                                                   "inner" \
                                                  ).drop(week2SalesAndFoodsSum['Customer ID'])

# Join food items to previous dataframe
customersFoodItems = customersWeek1And2Sales.join( \
                                                  week1And2Sales, \
                                                  customersWeek1And2Sales['ID'] == week1And2Sales['Customer ID'], \
                                                  "inner" \
                                                 )

customersFoodItems = customersFoodItems.drop('Customer ID')

In [29]:
customersFoodItems.show()

+---+----------+----------+------+-----------+--------------------+------------------+------------------+--------------------+
| ID|First Name| Last Name|Gender|    Company|          Occupation|Week 1 Bill amount|Week 2 Bill amount|          Food Items|
+---+----------+----------+------+-----------+--------------------+------------------+------------------+--------------------+
|919|    Ronald|     Welch|  Male|     Avamba|Help Desk Technician|              1.75|             11.25|      [Salad, Drink]|
|574|      Lisa|     Stone|Female|  Riffpedia|       Senior Editor|              1.75|              0.99|      [Drink, Donut]|
|462|      Anne|     Lopez|Female|       Mydo|Mechanical System...|              2.99|             11.25|       [Salad, Taco]|
|155|   Carolyn|      Diaz|Female|   Gigazoom|Database Administ...|              4.98|              2.99|[Taco, Sushi, Donut]|
|101|    Jeremy|    Nelson|  Male|Brainlounge|    Web Developer IV|             24.99|              4.25| [Ques

In [30]:
customersFoodItems.select('Food Items').where("ID == '77'").collect()

[Row(Food Items=['Burrito', 'Steak', 'Sushi', 'Donut'])]

In [31]:
# Find the Frequency of Ordered Food and Show them in descending

In [32]:
foods.show()

+-------+----------+-----+
|Food ID| Food Item|Price|
+-------+----------+-----+
|      1|     Sushi| 3.99|
|      2|   Burrito| 9.99|
|      3|      Taco| 2.99|
|      4|Quesadilla| 4.25|
|      5|     Pizza| 2.49|
|      6|     Pasta|13.99|
|      7|     Steak|24.99|
|      8|     Salad|11.25|
|      9|     Donut| 0.99|
|     10|     Drink| 1.75|
+-------+----------+-----+



In [33]:
from pyspark.sql.functions import count

orderedFoodCount = sales.groupBy('Food ID').agg(count('Food ID').alias('Frequency')).orderBy('Frequency', ascending = False)

In [34]:
orderedFoodCount.show()

+-------+---------+
|Food ID|Frequency|
+-------+---------+
|     10|       59|
|      2|       57|
|      9|       55|
|      6|       54|
|      7|       50|
|      4|       49|
|      8|       48|
|      3|       46|
|      1|       43|
|      5|       39|
+-------+---------+



In [35]:
foods.show()

+-------+----------+-----+
|Food ID| Food Item|Price|
+-------+----------+-----+
|      1|     Sushi| 3.99|
|      2|   Burrito| 9.99|
|      3|      Taco| 2.99|
|      4|Quesadilla| 4.25|
|      5|     Pizza| 2.49|
|      6|     Pasta|13.99|
|      7|     Steak|24.99|
|      8|     Salad|11.25|
|      9|     Donut| 0.99|
|     10|     Drink| 1.75|
+-------+----------+-----+



In [36]:
orderedFoodCount = orderedFoodCount.join(foods, orderedFoodCount['Food ID'] == foods['Food ID'], "left").drop(foods['Food ID'])

In [37]:
orderedFoodCount.show()

+-------+---------+----------+-----+
|Food ID|Frequency| Food Item|Price|
+-------+---------+----------+-----+
|     10|       59|     Drink| 1.75|
|      2|       57|   Burrito| 9.99|
|      9|       55|     Donut| 0.99|
|      6|       54|     Pasta|13.99|
|      7|       50|     Steak|24.99|
|      4|       49|Quesadilla| 4.25|
|      8|       48|     Salad|11.25|
|      3|       46|      Taco| 2.99|
|      1|       43|     Sushi| 3.99|
|      5|       39|     Pizza| 2.49|
+-------+---------+----------+-----+



In [38]:
# Lets Consider Colleagues came on team dinner and generate bill amount for a team(use company column in customer 
# sheet) combing Week1 and Week 2 in descending 
# Expected OP columns : Company Name|| Food Names || Bill Amount

In [39]:
result.show()

+---+----------+---------+------+------------+--------------------+-------+----------+-----+
| ID|First Name|Last Name|Gender|     Company|          Occupation|Food ID| Food Item|Price|
+---+----------+---------+------+------------+--------------------+-------+----------+-----+
|537|    Cheryl|  Carroll|Female|    Zoombeat|    Registered Nurse|      9|     Donut| 0.99|
| 97|    Amanda|  Watkins|Female|         Ozu| Account Coordinator|      4|Quesadilla| 4.25|
|658|   Patrick|     Webb|  Male|   Browsebug|Community Outreac...|      1|     Sushi| 3.99|
|202|     Louis| Campbell|  Male|   Rhynoodle|Account Represent...|      2|   Burrito| 9.99|
|155|   Carolyn|     Diaz|Female|    Gigazoom|Database Administ...|      9|     Donut| 0.99|
|213|     Keith|   Foster|  Male|   Gigashots|  VP Quality Control|      8|     Salad|11.25|
|600|    Sandra|   Bailey|Female|   Jaxnation|     Sales Associate|      1|     Sushi| 3.99|
|503|    Justin|  Daniels|  Male|        Oloo|  Environmental Tech|   

In [40]:
# Type cast Price column to double

result = result.withColumn('Price', result['Price'].cast('double'))
teamBillAmount = result.groupBy('Company') \
                       .agg( \
                            sum('Price').alias('Total Price'), \
                            collect_set('Food Item').alias('Food Items') \
                           ) \
                       .orderBy( \
                                'Total Price', \
                                ascending = False \
                               )

In [41]:
teamBillAmount.show()

+------------+------------------+--------------------+
|     Company|       Total Price|          Food Items|
+------------+------------------+--------------------+
|   Rhynoodle|              61.7|[Quesadilla, Burr...|
|       Yadel|55.959999999999994|[Taco, Steak, Pasta]|
|       Voomm|             54.96|[Quesadilla, Burr...|
|  Browseblab|54.470000000000006|[Pizza, Salad, St...|
|      Wikizz|54.209999999999994|[Salad, Taco, Ste...|
|Jabbersphere|53.199999999999996|[Burrito, Salad, ...|
|    Edgeblab|             49.98|             [Steak]|
|       Npath|49.220000000000006|[Burrito, Salad, ...|
|        Muxo|             48.97|[Burrito, Steak, ...|
|       Vimbo|47.489999999999995|      [Salad, Steak]|
|        Yata|             45.72|[Pizza, Quesadill...|
|   Bubblebox|             44.72|[Steak, Pasta, Dr...|
| Brainlounge|43.230000000000004|[Quesadilla, Stea...|
|    Fanoodle|             42.95|[Pizza, Burrito, ...|
|     Edgeify|              41.2|[Quesadilla, Past...|
|     Dabv

In [42]:
# Find who all Ordered same item and group them and find count of male and female in that, and what is over all value 
# that food items to the restaurant combining week 1 and week 2 sales
# Expected OP columns : Food Name|| Customer names || Count of male || count of Female || Overall Food Amount

In [53]:
result.show()

+---+----------+---------+------+------------+--------------------+-------+----------+-----+
| ID|First Name|Last Name|Gender|     Company|          Occupation|Food ID| Food Item|Price|
+---+----------+---------+------+------------+--------------------+-------+----------+-----+
|537|    Cheryl|  Carroll|Female|    Zoombeat|    Registered Nurse|      9|     Donut| 0.99|
| 97|    Amanda|  Watkins|Female|         Ozu| Account Coordinator|      4|Quesadilla| 4.25|
|658|   Patrick|     Webb|  Male|   Browsebug|Community Outreac...|      1|     Sushi| 3.99|
|202|     Louis| Campbell|  Male|   Rhynoodle|Account Represent...|      2|   Burrito| 9.99|
|155|   Carolyn|     Diaz|Female|    Gigazoom|Database Administ...|      9|     Donut| 0.99|
|213|     Keith|   Foster|  Male|   Gigashots|  VP Quality Control|      8|     Salad|11.25|
|600|    Sandra|   Bailey|Female|   Jaxnation|     Sales Associate|      1|     Sushi| 3.99|
|503|    Justin|  Daniels|  Male|        Oloo|  Environmental Tech|   

In [70]:
from pyspark.sql.functions import concat_ws

result = result.withColumn('Customer Name', concat_ws(' ','First Name','Last Name'))

In [71]:
result.show()

+---+----------+---------+------+------------+--------------------+-------+----------+-----+---------------+
| ID|First Name|Last Name|Gender|     Company|          Occupation|Food ID| Food Item|Price|  Customer Name|
+---+----------+---------+------+------------+--------------------+-------+----------+-----+---------------+
|537|    Cheryl|  Carroll|Female|    Zoombeat|    Registered Nurse|      9|     Donut| 0.99| Cheryl Carroll|
| 97|    Amanda|  Watkins|Female|         Ozu| Account Coordinator|      4|Quesadilla| 4.25| Amanda Watkins|
|658|   Patrick|     Webb|  Male|   Browsebug|Community Outreac...|      1|     Sushi| 3.99|   Patrick Webb|
|202|     Louis| Campbell|  Male|   Rhynoodle|Account Represent...|      2|   Burrito| 9.99| Louis Campbell|
|155|   Carolyn|     Diaz|Female|    Gigazoom|Database Administ...|      9|     Donut| 0.99|   Carolyn Diaz|
|213|     Keith|   Foster|  Male|   Gigashots|  VP Quality Control|      8|     Salad|11.25|   Keith Foster|
|600|    Sandra|   

In [81]:
from pyspark.sql.functions import when

result.groupBy('Food Item') \
        .agg( \
             collect_set('Customer Name').alias('Customers Name'), \
             count( \
                   when(result['Gender'] == 'Female', result['ID']) \
                  ).alias('Female Count'), \
             count( \
                   when(result['Gender'] == 'Male', result['ID']) \
                  ).alias('Male Count') \
            ).show()

+----------+--------------------+------------+----------+
| Food Item|      Customers Name|Female Count|Male Count|
+----------+--------------------+------------+----------+
|     Drink|[Doris Mcdonald, ...|          32|        27|
|     Salad|[Kathy Russell, A...|          27|        21|
|     Sushi|[Teresa Reynolds,...|          20|        23|
|     Steak|[Tammy Cole, Kath...|          28|        22|
|Quesadilla|[Teresa Reynolds,...|          23|        26|
|     Pizza|[Bobby Sims, Robe...|          18|        21|
|      Taco|[Melissa Mcdonald...|          27|        19|
|   Burrito|[Bobby Sims, Euge...|          28|        29|
|     Pasta|[Russell Garza, V...|          28|        26|
|     Donut|[Andrea Butler, R...|          35|        20|
+----------+--------------------+------------+----------+

